In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from datetime import datetime as dt
from datetime import timedelta
import os

In [10]:
app_dir = os.path.abspath('')
DATA_FILE = app_dir + '\\shiny_app\\data' + '\\database.xlsx'

advisors = pd.read_excel(DATA_FILE, sheet_name='advisors')
wash_list = pd.read_excel(DATA_FILE, sheet_name='wash_list')
countries = pd.read_excel(DATA_FILE, sheet_name='countries')
calendar = pd.read_excel(DATA_FILE, sheet_name='calendar')

In [11]:
countries.head()

,CIA Name,ISO 3166 alpha3,Continent,ta_focal,ta_support
0,Afghanistan,AFG,Asia,Ciaran,Katherine
1,Bangladesh,BGD,Asia,Ciaran,Katherine
2,Burkina Faso,BFA,Africa,Franck,NaN
3,Burundi,BDI,Africa,Franck,NaN
4,Central African Republic,CAF,Africa,Franck,NaN


In [12]:
cols = ['#4A628A', '#7AB2D3', '#B9E5E8', '#DFF2EB']
hover_data = {
    'ta_focal': False,
    'ISO 3166 alpha3': False,
    'Focal Point': countries.ta_focal,
}

fig = px.choropleth(
    data_frame=countries,
    locations='ISO 3166 alpha3',
    color='ta_focal',
    color_discrete_sequence=cols,
    hover_name='CIA Name',
    hover_data=hover_data,
    labels={'ta_focal': 'Focal Point'}
)

fig.show()

In [13]:
calendar.type.unique()

array(['Leave Full Day', 'Bank Holiday', 'Conference/Workshop',
       'Time In Lieu Full Day', 'Country Support Visit', 'Training',
       'Time In Lieu Half Day'], dtype=object)

In [14]:
def date_prettify(series):
    return f'{series:%d-%b-%Y}'

def create_color_col(series):
    if series=='Leave Full Day' or series=='Time In Lieu Full Day' or series=='Bank Holiday':
        hue = '#FA4032'
    elif series=='Leave Half Day' or series=='Time In Lieu Half Day':
        hue = '#FA812F'
    elif series=='Country Support Visit':
        hue='#FCC737'
    elif series=='Conference/Workshop' or series=='Training':
        hue='#FF2929'
    else:
        hue='#A6AEBF'
    
    return hue

# calendar['color'] = calendar.type.apply(create_color_col)
colors = [
    ('Leave Full Day','#FA4032'),('Time In Lieu Full Day','#FA4032'),('Bank Holiday','#FA4032'),
    ('Leave Half Day','#FA812F'),('Time In Lieu Half Day','#FA812F'),
    ('Conference/Workshop','#FF2929'),('Training','#FF2929')
]

hover_data={
    'start_date':False,
    'end_date':False,
    'advisor':False,
    'type':False,
    'from':calendar.start_date.apply(date_prettify),
    'to':calendar.end_date.apply(date_prettify),
    # 'Remarks':calendar.remarks if ~calendar.remarks.isna() else False
    # 'Country':calendar.remarks if calendar[calendar.type=='Country Support Visit'] else False
}

fig = px.timeline(
    data_frame=calendar,
    x_start='start_date',
    x_end='end_date',
    y='advisor',
    color='type',
    hover_name='type',
    hover_data=hover_data,
    text='remarks',
    range_x=[dt.today() - timedelta(15), dt.today() + timedelta(30)],
    labels={'type':''}
)
fig.update_traces(textposition='inside')
fig.update_xaxes(tickformat='%d-%b', dtick=86400000.0*5)

fig.show()

In [47]:
def total_busdays(data):
    return len(pd.bdate_range(data.start_date, data.end_date))

calendar['total_busdays'] = calendar.apply(total_busdays, axis=1)
calendar.groupby(['advisor']).agg({'total_busdays':'sum'})

,total_busdays
advisor,
Ciaran,18
Franck,18
Katherine,20
Paolo,14


In [42]:
year = 2024
grouped = calendar[calendar.end_date.dt.year <= year].groupby(['advisor', 'type']).agg({'total_busdays':'sum'}).reset_index()
yr_busdays = np.busday_count(str(year), str(year+1))
grouped['pcg_busdays'] = (grouped.total_busdays / yr_busdays)*100

grouped

,advisor,type,total_busdays,pcg_busdays
0,Ciaran,Bank Holiday,3,1.145038
1,Ciaran,Country Support Visit,11,4.198473
2,Ciaran,Leave Full Day,2,0.763359
3,Ciaran,Time In Lieu Full Day,1,0.381679
4,Franck,Bank Holiday,3,1.145038
5,Franck,Conference/Workshop,6,2.290076
6,Franck,Country Support Visit,6,2.290076
7,Franck,Leave Full Day,2,0.763359
8,Katherine,Bank Holiday,3,1.145038
9,Katherine,Country Support Visit,1,0.381679


In [52]:
fig = px.bar(
    grouped,#grouped[grouped.advisor=='Ciaran'],
    x='total_busdays',
    y='type',
    color='advisor',
    barmode='group'
)

fig.show()